In [1]:
import numpy as np
import plotly.express as px
import pandas as pd


In [2]:
df_ee = pd.read_excel("../data/processed/IDPiCombinatorialLibraryWithPredictions.xlsx")
"""
average_ee_by_name = df_ee.groupby('Imine')['ee'].mean().reset_index()
max_ee_by_name = df_ee.groupby(['Imine',"Catalyst_Ar_grp"])['Predicted ee'].max().reset_index()
max_ee_by_name = max_ee_by_name.loc[max_ee_by_name.groupby('Imine')['Predicted ee'].idxmax()]

average_ee_by_name = average_ee_by_name.rename(columns={'Imine': 'Name'})


max_ee_by_name = max_ee_by_name.rename(columns={'Imine': 'Name'})
max_ee_by_name = max_ee_by_name.rename(columns={'Predicted ee': 'ee_max'})

# Match the 'ee' column with the 'Experiment' column in your DataFrame based on name"""


'\naverage_ee_by_name = df_ee.groupby(\'Imine\')[\'ee\'].mean().reset_index()\nmax_ee_by_name = df_ee.groupby([\'Imine\',"Catalyst_Ar_grp"])[\'Predicted ee\'].max().reset_index()\nmax_ee_by_name = max_ee_by_name.loc[max_ee_by_name.groupby(\'Imine\')[\'Predicted ee\'].idxmax()]\n\naverage_ee_by_name = average_ee_by_name.rename(columns={\'Imine\': \'Name\'})\n\n\nmax_ee_by_name = max_ee_by_name.rename(columns={\'Imine\': \'Name\'})\nmax_ee_by_name = max_ee_by_name.rename(columns={\'Predicted ee\': \'ee_max\'})\n\n# Match the \'ee\' column with the \'Experiment\' column in your DataFrame based on name'

In [3]:
print(len(np.unique(df_ee["electrophile SMILES"])),len(np.unique(df_ee["nucleophile SMILES"])),len(np.unique(df_ee["catalyst SMILES"])))
unique_entries = df_ee["catalyst SMILES"].unique()
entry_to_id = {entry: id for id, entry in enumerate(unique_entries)}

# Create a new column 'ID' and use the map function to assign IDs
df_ee['ID'] = df_ee["catalyst SMILES"].map(entry_to_id)


df_ee["Reaction"]=df_ee["nucleophile SMILES"]+df_ee["electrophile SMILES"]
len(np.unique(df_ee["Reaction"]))


153 50 35


216

In [4]:
average_ee_by_name = df_ee.groupby('Reaction')['ee'].mean().reset_index()
max_ee_by_name = df_ee.groupby(['Reaction',"catalyst SMILES"])['ee'].max().reset_index()
max_ee_by_name = max_ee_by_name.loc[max_ee_by_name.groupby('Reaction')['ee'].idxmax()]

df_ee["rank"] = df_ee.groupby("Reaction")["ee"].rank(method="dense", ascending=False)


In [5]:

mean_rank = df_ee.groupby(['ID'])['rank'].mean()
print(mean_rank)

fig = px.bar(x=mean_rank.index, y=mean_rank,
                labels={'x':'Catalyst Class', 'y':'Rank Gen'}, title='Ranking',
                template='simple_white',
                width=980, height=720,
                )

fig.update_xaxes(mirror=True)
fig.update_yaxes(mirror=True)

fig.update_layout(
    font_size=20,
)

fig.show()
mean_rank.sort_values()

ID
0     29.912037
1     12.990741
2     34.189815
3     32.560185
4     19.416667
5     10.439815
6      7.518519
7     15.310185
8     27.037037
9     31.388889
10     2.782407
11    29.342593
12    24.851852
13    17.875000
14    10.777778
15     8.902778
16    13.472222
17    11.444444
18    17.708333
19     8.129630
20     2.898148
21    13.532407
22     2.314815
23    13.606481
24    28.287037
25    23.837963
26    18.907407
27    21.222222
28    21.907407
29    13.324074
30     6.023148
31    26.893519
32     6.199074
33    25.199074
34    25.023148
Name: rank, dtype: float64


ID
22     2.314815
10     2.782407
20     2.898148
30     6.023148
32     6.199074
6      7.518519
19     8.129630
15     8.902778
5     10.439815
14    10.777778
17    11.444444
1     12.990741
29    13.324074
16    13.472222
21    13.532407
23    13.606481
7     15.310185
18    17.708333
13    17.875000
26    18.907407
4     19.416667
27    21.222222
28    21.907407
25    23.837963
12    24.851852
34    25.023148
33    25.199074
31    26.893519
8     27.037037
24    28.287037
11    29.342593
0     29.912037
9     31.388889
3     32.560185
2     34.189815
Name: rank, dtype: float64

In [6]:
def calculate_var_es(data):
    var = np.percentile(data, 100 * (1 - 0.95))
    tail_losses = data[data < var]
    es = tail_losses.mean()
    return pd.Series({'VAR': var, 'ES': es})

results = df_ee.groupby('ID')['ee'].apply(calculate_var_es).reset_index()

fig = px.bar(results, x='ID', y='ee', title='VAR and ES by Asset Type',
             labels={'variable': 'Metric', 'value': 'Value'}, barmode='group',color='level_1')

# Customize the layout
fig.update_layout(xaxis_title='Asset Type', yaxis_title='Value', legend_title=None, xaxis_tickangle=-4,template='simple_white',width=980, height=720,
                  )

# Show the plot
fig.show()
results

,ID,level_1,ee
0,0,VAR,31.704442
1,0,ES,29.453820
2,1,VAR,76.560134
3,1,ES,74.770796
4,2,VAR,13.463195
...,...,...,...
65,32,ES,78.426297
66,33,VAR,66.016086
67,33,ES,63.686935
68,34,VAR,66.872638


In [7]:
df_ee["Quantile_rank"]= 0


df_ee['Quantile_rank'] = (
    df_ee.groupby('Reaction')['ee'].transform(lambda g: pd.qcut(g, q=20, labels=False))
)
mean_quant = df_ee.groupby(['ID'])['Quantile_rank'].mean()
print(mean_rank)

fig = px.bar(x=mean_quant.index, y=mean_quant,
                labels={'x':'Catalyst Class', 'y':'Rank Gen'},title='Quantiles',
                template='simple_white',
                width=980, height=720,
                )

fig.update_xaxes(mirror=True)
fig.update_yaxes(mirror=True)

fig.update_layout(
    font_size=20,
)

fig.show()
print(mean_quant.sort_values())
mean_rank.sort_values()
NEWDF = pd.DataFrame()
NEWDF['ee']= df_ee['ee']

NEWDF['Quantile_rank']= df_ee['Quantile_rank']
NEWDF['Reaction']= df_ee['Reaction']


ID
0     29.912037
1     12.990741
2     34.189815
3     32.560185
4     19.416667
5     10.439815
6      7.518519
7     15.310185
8     27.037037
9     31.388889
10     2.782407
11    29.342593
12    24.851852
13    17.875000
14    10.777778
15     8.902778
16    13.472222
17    11.444444
18    17.708333
19     8.129630
20     2.898148
21    13.532407
22     2.314815
23    13.606481
24    28.287037
25    23.837963
26    18.907407
27    21.222222
28    21.907407
29    13.324074
30     6.023148
31    26.893519
32     6.199074
33    25.199074
34    25.023148
Name: rank, dtype: float64


ID
2      0.032407
3      0.416667
9      1.435185
0      2.009259
11     2.416667
24     2.912037
8      3.796296
31     3.981481
33     4.879630
34     5.013889
12     5.148148
25     5.652778
28     6.879630
27     7.166667
4      8.421296
26     8.805556
13     9.300926
18     9.458333
7     10.828704
23    11.787037
21    11.912037
16    11.976852
29    12.157407
1     12.250000
17    13.125000
14    13.472222
5     13.810185
15    14.657407
19    15.199074
6     15.416667
32    16.282407
30    16.532407
10    18.078704
20    18.101852
22    18.337963
Name: Quantile_rank, dtype: float64


In [8]:
df_ee["Normalized_rank"]= 0


df_ee['Normalized_rank'] = (
    df_ee.groupby('Reaction')['ee'].transform(lambda g: (g-g.mean())/g.std())
)
mean_quant = df_ee.groupby(['ID'])['Normalized_rank'].mean()
print(mean_rank)

fig = px.bar(x=mean_quant.index, y=mean_quant,
                labels={'x':'Catalyst Class', 'y':'Rank Gen'}, title='Standardized',
                template='simple_white',
                width=980, height=720,
                )

fig.update_xaxes(mirror=True)
fig.update_yaxes(mirror=True)

fig.update_layout(
    font_size=20,
)

fig.show()
print(mean_quant.sort_values())
mean_rank.sort_values()
NEWDF = pd.DataFrame()
NEWDF['Normalized_rank']= df_ee['Normalized_rank']
NEWDF['Reaction']= df_ee['Reaction']

ID
0     29.912037
1     12.990741
2     34.189815
3     32.560185
4     19.416667
5     10.439815
6      7.518519
7     15.310185
8     27.037037
9     31.388889
10     2.782407
11    29.342593
12    24.851852
13    17.875000
14    10.777778
15     8.902778
16    13.472222
17    11.444444
18    17.708333
19     8.129630
20     2.898148
21    13.532407
22     2.314815
23    13.606481
24    28.287037
25    23.837963
26    18.907407
27    21.222222
28    21.907407
29    13.324074
30     6.023148
31    26.893519
32     6.199074
33    25.199074
34    25.023148
Name: rank, dtype: float64


ID
2    -3.100980
3    -1.626976
9    -1.472367
0    -1.175021
11   -1.105826
24   -0.931515
8    -0.888808
31   -0.806786
33   -0.451489
34   -0.429888
25   -0.349123
12   -0.322300
28    0.047928
27    0.052361
26    0.121907
4     0.159977
18    0.218226
13    0.239853
7     0.327292
29    0.387645
21    0.526024
23    0.530331
16    0.559878
1     0.583568
14    0.603236
17    0.624020
5     0.643122
15    0.678681
19    0.713884
6     0.715951
32    0.755737
30    0.848577
10    1.090687
20    1.113293
22    1.118901
Name: Normalized_rank, dtype: float64


In [11]:
df_ee["Normalized_rank"]= 0


df_ee['Normalized_rank'] = (
    df_ee.groupby('Reaction')['ee'].transform(lambda g: (g-g.min())/(g.max()-g.min()))
)
mean_quant = df_ee.groupby(['ID'])['Normalized_rank'].mean()
print(mean_rank)

fig = px.bar(x=mean_quant.index, y=mean_quant,
                labels={'x':'Catalyst Class', 'y':'Rank Gen'}, title='Standardized',
                template='simple_white',
                width=980, height=720,
                )

fig.update_xaxes(mirror=True)
fig.update_yaxes(mirror=True)

fig.update_layout(
    font_size=20,
)

fig.show()
print(mean_quant.sort_values())
mean_rank.sort_values()
NEWDF = pd.DataFrame()
NEWDF['Normalized_rank']= df_ee['Normalized_rank']
NEWDF['Reaction']= df_ee['Reaction']

ID
0     29.912037
1     12.990741
2     34.189815
3     32.560185
4     19.416667
5     10.439815
6      7.518519
7     15.310185
8     27.037037
9     31.388889
10     2.782407
11    29.342593
12    24.851852
13    17.875000
14    10.777778
15     8.902778
16    13.472222
17    11.444444
18    17.708333
19     8.129630
20     2.898148
21    13.532407
22     2.314815
23    13.606481
24    28.287037
25    23.837963
26    18.907407
27    21.222222
28    21.907407
29    13.324074
30     6.023148
31    26.893519
32     6.199074
33    25.199074
34    25.023148
Name: rank, dtype: float64


ID
2     0.004696
3     0.342960
9     0.378207
0     0.442945
11    0.460850
24    0.504704
8     0.508228
31    0.528514
33    0.623693
34    0.628764
12    0.647805
25    0.648364
28    0.736586
27    0.737630
26    0.753896
4     0.763228
18    0.777031
13    0.782649
7     0.803968
29    0.818468
21    0.857902
23    0.859088
16    0.866997
14    0.870236
1     0.873066
17    0.881219
5     0.887000
15    0.895494
19    0.903262
6     0.903423
32    0.912687
30    0.933176
10    0.987566
20    0.992357
22    0.993621
Name: Normalized_rank, dtype: float64


In [12]:
list = df_ee.columns.values.tolist()


['electrophile SMILES',
 'nucleophile SMILES',
 'catalyst SMILES',
 'MaxEStateIndexelectrophile',
 'MinEStateIndexelectrophile',
 'MaxAbsEStateIndexelectrophile',
 'MinAbsEStateIndexelectrophile',
 'qedelectrophile',
 'MolWtelectrophile',
 'HeavyAtomMolWtelectrophile',
 'ExactMolWtelectrophile',
 'NumValenceElectronselectrophile',
 'NumRadicalElectronselectrophile',
 'MaxPartialChargeelectrophile',
 'MinPartialChargeelectrophile',
 'MaxAbsPartialChargeelectrophile',
 'MinAbsPartialChargeelectrophile',
 'FpDensityMorgan1electrophile',
 'FpDensityMorgan2electrophile',
 'FpDensityMorgan3electrophile',
 'BCUT2D_MWHIelectrophile',
 'BCUT2D_MWLOWelectrophile',
 'BCUT2D_CHGHIelectrophile',
 'BCUT2D_CHGLOelectrophile',
 'BCUT2D_LOGPHIelectrophile',
 'BCUT2D_LOGPLOWelectrophile',
 'BCUT2D_MRHIelectrophile',
 'BCUT2D_MRLOWelectrophile',
 'BalabanJelectrophile',
 'BertzCTelectrophile',
 'Chi0electrophile',
 'Chi0nelectrophile',
 'Chi0velectrophile',
 'Chi1electrophile',
 'Chi1nelectrophile',
 'Ch